In [19]:
import pickle
from datasets import Dataset

def load_data_from_pickle(filepath):
    with open(filepath, 'rb') as f:
        data = pickle.load(f)
    return data['texts'],data['labels']

def prepare_dataset(texts, labels):
    ds = Dataset.from_dict({
        'texts': texts,
        'labels': labels,
    })
    return ds.train_test_split(test_size=0.2, shuffle=True)

In [20]:
dataset_id = 'dataset/addition_dataset_ct.pkl'
texts, labels = load_data_from_pickle(dataset_id)
dataset = prepare_dataset(texts, labels)

In [22]:
from transformers import AutoTokenizer

model_path = 'afmck/testing-llama-tiny'

tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token_id= tokenizer.eos_token_id

In [24]:
def preprocess_function(example):
    labels = example['labels']
    example = tokenizer(example['texts'], padding = True, truncation=True)
    example['labels'] = labels
    #print(example)
    return example

tokenized_dataset = dataset.map(preprocess_function)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [25]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [26]:
import evaluate
import numpy as np

clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

def sigmoid(x):
    return 1/(1 + np.exp(-x))

def compute_metrics(eval_pred):

    predictions, labels = eval_pred
    predictions = sigmoid(predictions)
    predictions = (predictions > 0.5).astype(int).reshape(-1)
    return clf_metrics.compute(predictions=predictions, references=labels.astype(int).reshape(-1))


In [27]:
print(tokenizer.vocab_size)
tokenizer.pad_token_id = tokenizer.eos_token_id
print(tokenizer.eos_token_id)
print(tokenizer.vocab_size)

32000
2
32000


In [33]:
'''
from transformers import LlamaForSequenceClassification, LlamaConfig
config = LlamaConfig(
        vocab_size=tokenizer.vocab_size,  # Based on the number of unique tokens
        hidden_size = 256,
        intermediate_size = 256,
        num_hidden_layers = 4,
        num_attention_heads = 4,
        max_position_embeddings = 512,
        use_cache = True,
        rope_theta=256,
        num_label = 30,
        
        pad_token_id = tokenizer.eos_token_id,
        bos_token_id = tokenizer.bos_token_id,
        eos_token_id = tokenizer.eos_token_id,
        
    )
model = LlamaForSequenceClassification(config)
'''
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    model_path, 
    num_labels=220, 
    #id2label=id2class, 
    #label2id=class2id, 
    problem_type = "multi_label_classification",
    pad_token_id = tokenizer.eos_token_id,
    
)
model.resize_token_embeddings(len(tokenizer))

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at afmck/testing-llama-tiny and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(32000, 512)

In [34]:
print(f"Total parameters in the model: {model.num_parameters()}")

Total parameters in the model: 37473792


In [35]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(

   output_dir="my_awesome_model",
   learning_rate=2e-5,
   per_device_train_batch_size=2,
   per_device_eval_batch_size=2,
   num_train_epochs=20,
   weight_decay=0.01,
   evaluation_strategy="epoch",
   save_strategy="epoch",
   load_best_model_at_end=True,
)

trainer = Trainer(

   model=model,
   args=training_args,
   train_dataset=tokenized_dataset["train"],
   eval_dataset=tokenized_dataset["test"],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [36]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.061300,0.061463,0.981450,0.898862,0.988230,0.824318
2,0.055400,0.055427,0.982493,0.904449,0.995630,0.828568
3,0.052300,0.052058,0.982936,0.907187,0.994552,0.833932
4,0.050200,0.050514,0.983357,0.910091,0.989693,0.842341
5,0.047500,0.049228,0.983784,0.912765,0.987748,0.848364
6,0.045800,0.048475,0.984309,0.915877,0.987208,0.854159
7,0.043500,0.047249,0.984575,0.917528,0.985873,0.858045
8,0.041100,1.650538,0.847266,0.155306,0.173739,0.140409


KeyboardInterrupt: 

In [38]:
trainer.evaluate(tokenized_dataset["test"])

{'eval_loss': 0.04566410183906555,
 'eval_accuracy': 0.9845227272727273,
 'eval_f1': 0.9172298118527882,
 'eval_precision': 0.9858135646358032,
 'eval_recall': 0.8575681818181818}

In [40]:
for i in range(6,21):
    print('Class: ', i)
    dataset_id = f'dataset/addition_dataset_ct{i}.pkl'
    texts, labels = load_data_from_pickle(dataset_id)
    test_dataset = prepare_dataset(texts, labels)
    tt_dataset = test_dataset.map(preprocess_function)
    print(trainer.evaluate(tt_dataset["test"]))

Class:  6


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'eval_loss': 0.22792428731918335, 'eval_accuracy': 0.9621181818181819, 'eval_f1': 0.7907922482176925, 'eval_precision': 0.8831015922852657, 'eval_recall': 0.7159545454545454}
Class:  7


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'eval_loss': 1.5584899187088013, 'eval_accuracy': 0.8560818181818182, 'eval_f1': 0.20387226552677895, 'eval_precision': 0.22813731007315702, 'eval_recall': 0.18427272727272728}
Class:  20


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'eval_loss': 1.6505376100540161, 'eval_accuracy': 0.847265909090909, 'eval_f1': 0.15530612501414043, 'eval_precision': 0.17373941899378498, 'eval_recall': 0.1404090909090909}
